In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
houses = ['A', 'B', 'C']
xtype_dict = {'r':'raw', 'c':'change'}

In [3]:
def read_data(house, xtype):
    assert house in houses
    assert xtype in xtype_dict
    
    act_df = pd.read_csv("house{}_act.csv".format(house))
    sensor_df = pd.read_csv("house{}_sensor.csv".format(house))
    X = np.load("X_{}_house{}.npy".format(xtype_dict[xtype], house))
    Y = np.load("Y_house{}.npy".format(house))
    return act_df, sensor_df, X, Y

In [20]:
#read in data
house = 'B'
t = 'r'
act_df,sensor_df,X,Y = read_data(house, t)

In [21]:
#build 4 dict, act to idx, idx to act, sensor to idx, idx to sensor
all_act = list(set(act_df.label)) + [0] #all the act labels
a2i = dict(zip(all_act, range(len(all_act))))
i2a = dict(zip(range(len(all_act)), all_act))

all_sensor = list(set(sensor_df.label)) #all the sensor labels
i2s = dict(zip(range(len(all_sensor)), all_sensor))
s2i = dict(zip(all_sensor, range(len(all_sensor))))

In [22]:
print "# sensors: ", len(all_sensor)
print "# states/acts: ", len(all_act)
print "# timeframes: ", len(X)

# sensors:  22
# states/acts:  25
# timeframes:  38900


## Naive Bayes

In [23]:
#each row represents sensors for a specific action i
#each column is sensor j
def calc_miu(X, Y, all_sensor=all_sensor, all_act=all_act):
    assert len(X)==len(Y)
    
    num_act = len(all_act)
    num_sensor = len(all_sensor)
    miu = np.zeros([num_act, num_sensor])
    for act in all_act:
        sensor = X[Y==act]
        for row in sensor:
            miu[a2i[act]] += row
    #normalize
    for i,row in enumerate(miu):
        total = np.sum(row)
        if total != 0:
            miu[i] = miu[i]/total
        
    return miu

In [24]:
def max_like(miu, X):
    num_time = len(X)
    num_act = len(miu)
    
    prob = np.zeros((num_time, num_act))
    for i in range(num_time):
        sensor = X[i]
        for j in range(num_act):
            act = miu[j]
            logp = 0
            for k,s in enumerate(sensor):
                if s == 1:
                    logp += np.log(act[k])
                else:
                    logp += np.log(1-act[k])
            prob[i,j] = logp
    return prob

In [25]:
def mle(matrix):
    est = np.zeros(len(matrix))
    for i,row in enumerate(matrix):
        est[i] = np.argmax(row)
    return est

In [26]:
miu = calc_miu(X,Y)
likes = max_like(miu,X)
result = mle(likes) #note: result is index, not label yet

In [27]:
#convert index to label
res_label = np.array([i2a[e] for e in result])
for act in all_act:
    print "act label: {}, Y count:{}, estimate count:{}".format(act, sum(Y==act), sum(res_label==act))

act label: 1.0, Y count:12242, estimate count:326
act label: 4.0, Y count:49, estimate count:651
act label: 5.0, Y count:115, estimate count:1019
act label: 6.0, Y count:30, estimate count:53
act label: 9.0, Y count:9, estimate count:56
act label: 10.0, Y count:6057, estimate count:2940
act label: 11.0, Y count:28, estimate count:931
act label: 13.0, Y count:85, estimate count:51
act label: 15.0, Y count:90, estimate count:128
act label: 17.0, Y count:6, estimate count:345
act label: 24.0, Y count:27, estimate count:2732
act label: 29.0, Y count:30, estimate count:1789
act label: 31.0, Y count:50, estimate count:932
act label: 32.0, Y count:137, estimate count:2593
act label: 33.0, Y count:35, estimate count:1306
act label: 34.0, Y count:38, estimate count:1291
act label: 35.0, Y count:77, estimate count:891
act label: 36.0, Y count:4, estimate count:362
act label: 37.0, Y count:14, estimate count:255
act label: 38.0, Y count:3, estimate count:98
act label: 40.0, Y count:475, estimate 

# Evaluation

$$\text{Precision} = \frac{1}{N}\sum^N_i \frac{TP_i}{TI_i} $$
$$\text{Recall} = \frac{1}{N}\sum^N_i\frac{TP_i}{TT_i} $$
$$\text{F-Measure} = \frac{2 \cdot precision \cdot recal}{precision+recall} $$
$$\text{Accuracy} = \frac{\sum^N_i TP_i}{Total}$$

In [28]:
def precision(pred_label, Y):
    all_label = list(set(Y))
    N = len(all_label)
    res = 0
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        TI = np.sum(pred_label==y)
        if TI != 0:
            res += (float(TP)/TI)
    return float(res)/N

def recall(pred_label, Y):
    all_label = list(set(Y))
    N = len(all_label)
    res = 0
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        TT = np.sum(Y==y)
        if TT != 0:
            res += float(TP)/TT
    return float(res)/N

def f_score(pred_label, Y):
    p = precision(pred_label, Y)
    r = recall(pred_label, Y)
    return 2*p*r/(p+r)

def accuracy(pred_label, Y):
    res = 0
    all_label = list(set(Y))
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        res += TP
    return float(res)/len(Y)

In [29]:
precision(res_label, Y)

0.153465015209435

In [30]:
recall(res_label, Y)

0.5671278520032389

In [31]:
f_score(res_label, Y)

0.2415629917904369

In [32]:
accuracy(res_label, Y)

0.41439588688946016

In [33]:
#save miu
np.save("mu{}.npy".format(house), miu)